## Описание нового алгоритма парсинга сайта [nemez1da.ru](https://nemez1da.ru/)

### Навигация
+ [Описание](#описание)
+ [Библиотеки](#библиотеки)
+ [Алгоритм поиска ссылок](#алгоритм-поиска-ссылок)
+ [Алгоритм поиска конечных данных](#алгоритм-поиска-конечных-данных)
+ [Примеры](#примеры)

### Описание
Инструменты парсинга соответствуют следущим критериям:
1. Используется однопоточный режим обработки данных на всем промежутке процесса парсинга
2. Для получения ссылок используется рекурсия
3. Алгоритм сперва находит ссылки, только потом данные о каждом человеке


### Библиотеки

Парсинг инструментами BeautifulSoup4 и lxml

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json

Будем использовать сессию

In [2]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import requests


def get_session() -> requests.Session:
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

### Алгоритм поиска ссылок

In [3]:
def get_links(self, url):
    soup_object = BeautifulSoup(get_session().get(url, headers=self.__headers).text, 'lxml')
    person_links = soup_object.find_all('h3', {'class': 'simple-grid-grid-post-title'})
    links = [elem.find('a').get('href') for elem in person_links]
    self.links.extend(links)
    print(links)
    try:
        next_page = soup_object.find('a', {'class': 'next page-numbers'}).get('href')
    except AttributeError:
        return
    else:
        get_links(next_page)

### Алгоритм поиска конечных данных

In [4]:
def get_person_data(url: str, headers: dict):
    data_frame, photo = dict(), list()
    try:
        soup_object = BeautifulSoup(get_session().get(url, headers=headers).text, 'lxml')
        data_frame['ФИО'] = [soup_object.title.text.split(" - ")[0]]
        for elem in soup_object.find_all('div' and 'b'):
            key, value = elem.parent.find('b').text, elem.parent.text
            data_frame[key] = [value.replace(key, '').strip()]
        data_frame['Сайт'] = [url]
        for link in soup_object.find_all('a'):
            actual = link.get('href')
            if str(actual).endswith('.jpg'):
                photo.append(str(actual))
        data_frame['Фото'] = json.dumps(photo)
    except Exception as ex:
        return ex
    else:
        return pd.DataFrame.from_dict(data_frame)

#### В версии телеграм бота используется урезаная версия парсера конечных данных для создания фиксированных столбцов и удобной навигации по ним, оастльная же информация подргружается в процессе детального поиска

In [5]:
def get_person_data_lite(url: str, headers: dict):
    data_frame = dict()
    try:
        soup_object = BeautifulSoup(get_session().get(url, headers=headers).text, 'lxml')
        data_frame['full_name'] = [soup_object.title.text.split(" - ")[0]]
        for elem in soup_object.find_all('div' and 'b'):
            key, value = elem.parent.find('b').text, elem.parent.text
            if key == 'Дата рождения':
                data_frame['date_of_birth'] = [value.replace(key, '').strip()]
            elif key == 'Категория':
                data_frame['category'] = [value.replace(key, '').strip()]
        if 'date_of_birth' not in data_frame:
            data_frame['date_of_birth'] = ['Нет информации']
        if 'category' not in data_frame:
            data_frame['category'] = ['Нет информации']
        data_frame['link'] = [url]
    except Exception as ex:
        return ex
    else:
        return pd.DataFrame.from_dict(data_frame)

### В версии телеграм бота так же реализован парсинг "На лету" за счет имеющейся функции поиска на самом сайте. Так, мы можем создать запрос и уже потом распарсить сам ответ

создадим конструктор ссылок

In [12]:
def query_creator(query: str) -> str:
    return 'https://nemez1da.ru/?s=' + query.replace(' ', '+')

метод получения ссылок

In [13]:
def get_links(url: str, headers: dict):
    url1, links = query_creator(url), list()
    soup = BeautifulSoup(get_session().get(url1, headers=headers).text, 'lxml')
    person_links = soup.find_all('h3', {'class': 'simple-grid-grid-post-title'})
    for link in person_links:
        url = ''.join([i.lower() for i in url if not i.isdigit() and not i == ' '])
        if url in link.text.lower().replace(' ', ''):
            links.append(link.find('a').get('href'))
    return links

In [ ]:
get_links('Пермяков', headers=headers)

In [6]:
links = list()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'}
test_person_url = 'https://nemez1da.ru/voennye-prestupniki/vsu/voenkomy-tczk/chvertko-aleksandr-valerevich-chvertko-oleksandr-valerijovich/'
test_person_url_2 = 'https://nemez1da.ru/posobniki/network-terrorists/kabachij-roman-ivanovich-kabachij-roman-ivanovich/'
test_link_tree = 'https://nemez1da.ru/voennye-prestupniki/'

In [7]:
get_person_data(url=test_person_url, headers=headers)

,ФИО,Дата рождения,Должность,Проживает по адресу,ДРФО,Паспорт,Телефон,Почта,Вконтакте,Сайт,Фото
0,Чвертко Александр Валерьевич (Чвертко Олександ...,21.08.1986,Сотрудник Киевского городского ТЦК и СП,"Украина, Черниговская обл., г. Чернигов, ул. А...",3164402994,НМ152061,380442235013380933412399380937276037,glyuz.mariya@mail.ru,https://vk.com/id326711973https://vk.com/id965...,https://nemez1da.ru/voennye-prestupniki/vsu/vo...,"[""https://nemez1da.ru/wp-content/uploads/2024/..."


In [8]:
get_person_data(url=test_person_url_2, headers=headers)

,ФИО,Дата рождения,Категория,Зарегистрирован по адресу,Проживает по адресу,ДРФО,Паспорт,Телефон,Телеграм,Почта,Деятельность,Дополнительно,Сайт,Фото
0,Кабачий Роман Иванович (Кабачій Роман Іванович),03.03.1978,Сетевые террористы,"Украина, Киевская обл., г. Буча, ул. Энергетик...","Украина, Херсонская обл., г. Херсон, ул. Литей...",2855118519,СТ448243,"380635747085, 380666327703",1777885182,"romisz_k@yahoo.de, r.kabachiy@gmail.com","Украинский историк и журналист, окончил аспира...",Предоставлял ложную информацию для Польских СМ...,https://nemez1da.ru/posobniki/network-terroris...,"[""https://nemez1da.ru/wp-content/uploads/2023/..."


In [9]:
get_person_data_lite(url=test_person_url, headers=headers)

,full_name,date_of_birth,category,link
0,Чвертко Александр Валерьевич (Чвертко Олександ...,21.08.1986,Нет информации,https://nemez1da.ru/voennye-prestupniki/vsu/vo...


In [10]:
get_person_data_lite(url=test_person_url_2, headers=headers)

,full_name,date_of_birth,category,link
0,Кабачий Роман Иванович (Кабачій Роман Іванович),03.03.1978,Сетевые террористы,https://nemez1da.ru/posobniki/network-terroris...


In [17]:
touch_and_parse = get_links("Груша", headers=headers)
touch_and_parse

['https://nemez1da.ru/voennye-prestupniki/svr-ukrainy/grusha-tatyana-yurevna-grusha-tetyana-yuri%d1%97vna/',
 'https://nemez1da.ru/voennye-prestupniki/svr-ukrainy/grusha-tatyana-yurevna-grusha-tetyana-yuri%d1%97vna-2/',
 'https://nemez1da.ru/voennye-prestupniki/sotrudniki-sbu/sbu/grusha-dmitrij-nikolaevich-grusha-dmitro-mikolajovich/']

Вот так мы на лету получили все совпадения, теперь можно применить либой из методов парсинга странички

In [18]:
get_person_data(url=touch_and_parse[0], headers=headers)

,ФИО,Дата рождения,Категория,Должность,Звание,Телефон,Телеграм,Деятельность,Сайт,Фото
0,ГРУША Татьяна Юрьевна ГРУША Тетяна Юріївна,12.01.1969,СВР Украины,специалист 1 категории,майор,380994369184380962587258,409610186,"СВР Украины, ВАОЗІ 2 департамента",https://nemez1da.ru/voennye-prestupniki/svr-uk...,[]
